In [1]:
"""
Created on Wed Apr 20 16:59:21 2022

@author: Evan Shapiro
"""
import numpy as np
import copy as copy
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.ops import unary_union
import util
import seaborn as sns
import numpy as np
from matplotlib import cm, colors
import matplotlib.pyplot as plt

# Enable OSMNX cache
ox.config(use_cache=True)

### Import a network  of Denver streets that are both walkable and driveable using OSMnx.

In [3]:
denver = ox.graph_from_place("Denver, Colorado, USA", network_type="drive")

C:\Users\Evan Shapiro\Documents\Python\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


### Adding traffic capacities to all streets in Denver.

In [4]:
denver = util.add_speeds_to_graph(denver)
denver = util.drive_network_to_capacitated_network(denver)

In [34]:
for u,v, attr in denver.edges:
    print(denver.edges[u,v,attr]['capacity'])


5065.628401802529
5585.912619614762
2031.114761951746
3723.941746409841
3723.941746409841
1287.804939181203
2575.609878362406
5151.219756724812
1487.4543392961573
1487.4543392961573
3863.4148175436094
1287.804939181203
4535.541660181126
1487.4543392961573
4052.5027214420234
1015.557380975873
3039.3770410815177
2031.114761951746
3477.0025720389585
1487.4543392961573
1487.4543392961573
1487.4543392961573
1015.557380975873
1015.557380975873
1013.1256803605058
2318.001714692639
1013.1256803605058
2318.001714692639
1015.557380975873
2031.114761951746
2026.2513607210117
4114.004967604377
3477.0025720389585
1403.6617014383387
1487.4543392961573
4210.985104315016
2318.001714692639
1487.4543392961573
1403.6617014383387
2318.001714692639
1287.804939181203
4210.985104315016
1403.6617014383387
1403.6617014383387
1056.7997888335249
1287.804939181203
2113.5995776670497
1585.1996832502873
2113.5995776670497
1487.4543392961573
1487.4543392961573
1403.6617014383387
1403.6617014383387
1487.4543392961573

1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
2603.604614217989
1487.4543392961573
1306.4287754929571
3467.9105160310237
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
2603.604614217989
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
5151.219756724812
7018.308507191694
2974.9086785923146
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1287.804939181203
1487.4543392961573
1487.4543392961573
14

1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
1487.4543392961573
3477.0025720389585
1159.0008573463194
1287.804939181203
2318.001714692639
2318.001714692639
1403.6617014383387
1487.4543392961573
1403.6617014383387
5151.219756724812
1487.4543392961573
1487.4543392961573
4462.363017888472
1487.4543392961573
1487.4543392961573
2318.001714692639
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
4114.004967604377
1487.4543392961573
1487.4543392961573
1487.4543392961573
14

In [5]:
def Adj_f_Res(G):            
    for s, nbrs in G.adjacency():
            #print(nbrs,':')
            G.nodes[s]['Adj'] = []
            for t, data in nbrs.items():
                G.nodes[s]['Adj'].append(t)
                #print(G.nodes[s]['Adj'])
    return G

### Given a graph, define a residual network graph R to work on.

In [164]:
def Res(G):
    R = nx.DiGraph()
    #R = G.copy()
    R.add_nodes_from(G)
    print(R.nodes)
    #R.add_edges_from(G)

    #inf = float("inf")
    # Extract edges with positive capacities. Self loops excluded.
    edge_list = [
        (u, v, attr)
        for u, v, attr in G.edges(data=True)
        if u != v
    ]
    for u,v in G.edges:
        r = G.edges[u,v]['capacity'] 
        if not R.has_edge(u,v):
            R.add_edge(u,v)
            R[u][v]['capacity'] = r
            R.add_edge(v,u)
            R[v][u]['capacity'] = 0
        else:
            R[u][v]['capacity']=r
    R = Adj_f_Res(G,R)
    for u in R.nodes:
        R.nodes[u]['excess'] = 0
    for u,v in R.edges:
        R[u][v]['flow'] = 0
    return R      

In [6]:
def Res(G):
    for u,v in G.edges:
        print(G[u][v])
        if not G.has_edge(v,u):
            G.add_edge(v,u)
            G.edges[v,u]['capacity'] = 0
            
    G = Adj_f_Res(G)
    
    for u in G.nodes:
        G.nodes[u]['excess'] = 0
    for (u,v) in G.edges:
        G.edges[(u,v)]['flow'] = 0
    return G  

In [43]:
for u, v, i in denver.edges:
        print(denver[u][v][i]['capacity'])
            
        

5065.628401802529
5585.912619614762
2031.114761951746
3723.941746409841
3723.941746409841
1287.804939181203
2575.609878362406
5151.219756724812
1487.4543392961573
1487.4543392961573
3863.4148175436094
1287.804939181203
4535.541660181126
1487.4543392961573
4052.5027214420234
1015.557380975873
3039.3770410815177
2031.114761951746
3477.0025720389585
1487.4543392961573
1487.4543392961573
1487.4543392961573
1015.557380975873
1015.557380975873
1013.1256803605058
2318.001714692639
1013.1256803605058
2318.001714692639
1015.557380975873
2031.114761951746
2026.2513607210117
4114.004967604377
3477.0025720389585
1403.6617014383387
1487.4543392961573
4210.985104315016
2318.001714692639
1487.4543392961573
1403.6617014383387
2318.001714692639
1287.804939181203
4210.985104315016
1403.6617014383387
1403.6617014383387
1056.7997888335249
1287.804939181203
2113.5995776670497
1585.1996832502873
2113.5995776670497
1487.4543392961573
1487.4543392961573
1403.6617014383387
1403.6617014383387
1487.4543392961573

1287.804939181203
1287.804939181203
1487.4543392961573
1287.804939181203
1287.804939181203
1287.804939181203
1487.4543392961573
1287.804939181203
1287.804939181203
3863.4148175436094
1931.7074087718047
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
743.7271696480786
743.7271696480786
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1487.4543392961573
1487.4543392961573
1287.804939181203
1287.804939181203
1287.804939181203
1287.804939181203
2807.3234028766774
1287.

1487.4543392961573
4114.004967604377
4636.003429385278
2742.6699784029183
3477.0025720389585
5485.339956805837
1487.4543392961573
1487.4543392961573
1403.6617014383387
1403.6617014383387
1487.4543392961573
4114.004967604377
1487.4543392961573
1487.4543392961573
2742.6699784029183
1487.4543392961573
1403.6617014383387
1487.4543392961573
1403.6617014383387
1487.4543392961573
4114.004967604377
2231.181508944236
1487.4543392961573
1507.722763785973
1507.722763785973
2231.181508944236
2231.181508944236
2231.181508944236
1507.722763785973
4523.168291357919
1487.4543392961573
1403.6617014383387
1403.6617014383387
1487.4543392961573
1403.6617014383387
1487.4543392961573
1403.6617014383387
1403.6617014383387
1487.4543392961573
1403.6617014383387
1403.6617014383387
1403.6617014383387
1487.4543392961573
5207.209228435978
1159.0008573463194
4636.003429385278
2742.6699784029183
4636.003429385278
2031.114761951746
2031.114761951746
3477.0025720389585
1287.804939181203
1301.8023071089945
3477.0025720

In [ ]:
def drive_network_to_capacitated_network(G: nx.DiGraph, method=capacity_moore):
    """Add capacities to the network G using a particular method"""
    G = G.copy()
    for u, v, i in G.edges:
        edge_data = G.get_edge_data(u, v, i)
        raw_lanes = edge_data.get("lanes")
        if raw_lanes is None:
            lanes = 1
        elif isinstance(raw_lanes, str):
            lanes = int(raw_lanes)
            if not edge_data["oneway"]:
                lanes /= 2
        elif isinstance(raw_lanes, list):
            lanes = next(int(x) for x in raw_lanes)
        else:
            raise ValueError(f"No condition for lanes as {type(raw_lanes)}")

        edge_data["capacity"] = method(lanes, edge_data["speed_kph"])

    return G

### Defining adjacency lists for all nodes in the residual network.

### Defining a breadth search first algorithm on the residual network.

In [7]:
def BFS_res(R, s):
    
    if s not in R.nodes:
        print(s, 'is not a valid node')
        return   
    R.nodes[s]['D'] = 0
    pred = dict()
    pred[s] = 0
    visited_nodes = [s] #Keep track of visited nodes to not make incorrect distance labels
    current_node = [s]
    while current_node:
        n= current_node.pop(0)
        if R.nodes[n]['Adj']: #Check if i is a key value in the adjacency list
            for i in range(len(R.nodes[n]['Adj'])):
                j = R.nodes[n]['Adj'][i]
                if j not in visited_nodes:
                    visited_nodes.append(j)                  
                    pred[j] = n
                    current_node.append(j)
                    R.nodes[j]['D'] = R.nodes[n]['D'] +1
    return R

### Preprocessing the residual network for the Pre-Flow Push Max-Flow Algorithm

In [8]:
def preprocess_res(R, s):
    
    if s not in R.nodes:
        print(s, 'is not a valid node')
        return
    
    R.nodes[s]['D'] = len(R.nodes) #Set distance label of sink to |N|
    
    adj_s = R.nodes[s]['Adj'] 
    active = []
    for j in adj_s:
        R[s][j]['flow'] = R[s][j]['capacity'] 
        R.nodes[j]['excess'] = R[s][j]['capacity'] 
        if R.nodes[j]['excess'] >0:
            active.append(j)
    
    return R, active

### Defining Push Relabel Component of the Pre-Flow Push Algorithm

In [10]:
def push_relabel_res(R, i, active_nodes): #Adj_L,D,e,
    adj_l= copy.deepcopy(R.nodes[i]['Adj'])
    cur_dis = []
    for j in adj_l:
        cur_dis.append(R.nodes[j]['D'])
    #Given a node (i), we check to see if there is an admissible arc (i,j) in the residual network
    #If there is, push flow across the edge.
    if R[i]['D'] +1 in cur_dis:
        #Check to see if (i,j) is in the residual network
        #if (i,j,_) in G.edges and G.nodes[i]['D'] == G.nodes[j]['D'] +1:
        delta = min(R[i]['excess'],R[i][j]['capacity']) #G.edges[(i,j)]['Capacity']) #choose a
        
        #Update residual network
        #Check if G.nodes[i]['Excess'] exists. If it does ad delta flow to
        #the excess
        if R.nodes[i]['excess']:
            R[i]['excess'] -= delta
        else:
            R.nodes[i]['excess'] = -delta
            
        if R.nodes[j]['excess']:
            R.nodes[j]['excess'] += delta
        else:
            R.nodes[j]['excess'] = delta
            
      
        R[i][j]['capacity'] -= delta
        R[j][i]['capacity'] += delta
        #if i not in G[j]['Adj_l']:
            #G[j]['Adj_L'].append(i)
        
        #Do we need to explicitly remove arcs whose residual capacity is 0
    #If there is no admissible arc (i,j) 
    else:
        while cur_dis:
            #Construct a set of all distance labels for nodes in the adjacency 
            #list of node i
            #G.nodes[i]['D'] = [G.nodes[j]['D'] +1: (i,j) in G.edges and G.edges[(i,j)]['R_cap'] >0 ]
            d_min = min(cur_dis)
            index = cur_dis.index(d_min)
            upd_node = adj_l[index] #G.nodes[i]['Adj'][index]
            if R[i][upd_node]['capacity'] > 0:
                R[i]['D'] = R[upd_node]['D'] + 1
            else:
                cur_dis.pop(index)
                adj_l.pop(index)
    return active_nodes, R

In [35]:
def push_relabel_res(R, i, active_nodes): #Adj_L,D,e,
    adj_l= copy.deepcopy(R.nodes[i]['Adj'])
    cur_dis = []
    rec_node = []
    flows = {}
    while R.nodes[i]['excess'] > 0:   
        for j in adj_l:
            print(R.nodes[i]['excess'])
            cur_dis.append(R.nodes[j]['D'])
        #Given a node (i), we check to see if there is an admissible arc (i,j) in the residual network
        #If there is, push flow across the edge.
            if R.nodes[i]['D'] == R.nodes[j]['D'] +1 and R[i][j]['capacity'] >0:
                active_nodes.append(j)
                #Check to see if (i,j) is in the residual network
                #if (i,j,_) in G.edges and G.nodes[i]['D'] == G.nodes[j]['D'] +1:
                delta = min(R.nodes[i]['excess'],R[i][j]['capacity']) #G.edges[(i,j)]['Capacity']) #choose a
                
                #Update residual network
                #Check if G.nodes[i]['Excess'] exists. If it does ad delta flow to
                #the excess
                R.nodes[i]['excess'] = -delta
                    
                if R.nodes[j]['excess']:
                    R.nodes[j]['excess'] += delta
                else:
                    R.nodes[j]['excess'] = delta
                    
                if flows[(i,j)]:
                    flows[(i,j)]+= delta
                else:
                    flows[(i,j)] = delta
                    
              
                R[i][j]['capacity'] -= delta
                R[j][i]['capacity'] += delta
                #if i not in G[j]['Adj_l']:
                    #G[j]['Adj_L'].append(i)
                
                #Do we need to explicitly remove arcs whose residual capacity is 0
            #If there is no admissible arc (i,j) 
            elif R.nodes[i]['D'] < R.nodes[j]['D'] +1 and  R[i][j]['capacity'] >0:
                rec_node.append(j)
            
        if rec_node:
            print('yes')
            #print(R.nodes[i]['excess'])
                    #Construct a set of all distance labels for nodes in the adjacency 
                    #list of node i
                    #G.nodes[i]['D'] = [G.nodes[j]['D'] +1: (i,j) in G.edges and G.edges[(i,j)]['R_cap'] >0 ]
            dis = []
            for j in rec_node:
                    dis.append(R.nodes[j]['D'])
            d_min = min(dis)
            index = dis.index(d_min)
            j = rec_node[index]
            active_nodes.append(j)
            R.nodes[i]['D'] = d_min + 1
            delta = min(R.nodes[i]['excess'],R[i][j]['capacity']) #G.edges[(i,j)]['Capacity']) #choose a
                
                #Update residual network
                #Check if G.nodes[i]['Excess'] exists. If it does ad delta flow to
                #the excess
            if R.nodes[i]['excess']:
                R.nodes[i]['excess'] -= delta
            else:
                R.nodes[i]['excess'] = -delta
                
            if R.nodes[j]['excess']:
                R.nodes[j]['excess'] += delta
            else:
                R.nodes[j]['excess'] = delta
                
            if flows[(i,j)]:
                    flows[(i,j)]+= delta
            else:
                    flows = {}
                    flows[(i,j)] = delta  
          
            R[i][j]['capacity'] -= delta
            R[j][i]['capacity'] += delta
            rec_node.pop(0)
            print(rec_node)
            # else:
            #     cur_dis.pop(index)
            #     adj_l.pop(index)
    return active_nodes, R

In [33]:
def pre_flow_push(G,s,t):
    G = Res(G)
    G = Adj_f_Res(G)
    print(G.nodes[1])
    G = BFS_res(G,t)
    G, active_nodes = preprocess_res(G,s)
    while active_nodes:
        i = active_nodes.pop(0)
        print(i)
        if G.nodes[i]['excess'] > 0:
            active_nodes, G  = push_relabel_res(G, i, active_nodes)
            #print(active_nodes)
    print(G.nodes[t]['excess'])
    return G
    
    #Need to add nodes to active node list in push relabel algorithm

In [12]:
from unittest import TestCase
class TestPreflowPush(TestCase):
    """Tests for preflow_push"""

    def test_simple(self):
        """Test a simple graph for max-flow"""
        graph = nx.DiGraph()

        graph.add_edges_from(
            [
                (0, 1, {"capacity": 15}),
                (0, 3, {"capacity": 4}),
                (1, 2, {"capacity": 12}),
                (2, 3, {"capacity": 3}),
                (2, 5, {"capacity": 7}),
                (3, 4, {"capacity": 10}),
                (4, 1, {"capacity": 5}),
                (4, 5, {"capacity": 10}),
            ]
        )

        flow_value, flows = pre_flow_push(graph, 0, 5)

        self.assertEqual(flow_value, 14)
        self.assertEqual(
            flows,
            {
                (0, 1): 10,
                (0, 3): 4,
                (2, 3): 3,
                (1, 2): 10,
                (2, 5): 7,
                (4, 1): 0,
                (4, 5): 7,
                (3, 4): 7,
            },
        )

In [36]:
import unittest
unittest.main(argv=[''], verbosity=2, exit=False)

test_simple (__main__.TestPreflowPush)
Test a simple graph for max-flow ... ERROR

ERROR: test_simple (__main__.TestPreflowPush)
Test a simple graph for max-flow
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Evan Shapiro\AppData\Local\Temp\ipykernel_2456\2785659467.py", line 22, in test_simple
    flow_value, flows = pre_flow_push(graph, 0, 5)
  File "C:\Users\Evan Shapiro\AppData\Local\Temp\ipykernel_2456\688214651.py", line 11, in pre_flow_push
    active_nodes, G  = push_relabel_res(G, i, active_nodes)
  File "C:\Users\Evan Shapiro\AppData\Local\Temp\ipykernel_2456\1537756572.py", line 28, in push_relabel_res
    if flows[(i,j)]:
KeyError: (1, 2)

----------------------------------------------------------------------
Ran 1 test in 0.006s

FAILED (errors=1)


{'capacity': 15}
{'capacity': 4}
{'capacity': 12}
{'capacity': 0}
{'capacity': 10}
{'capacity': 0}
{'capacity': 3}
{'capacity': 7}
{'capacity': 0}
{'capacity': 0}
{'capacity': 5}
{'capacity': 10}
{'capacity': 0}
{'Adj': [2, 0, 4], 'excess': 0}
1
15


In [107]:
R= Res(denver)
for u in R.nodes:
    print(u)

27512439
27548618
27548632
27548648
89492109
89492111
89675931
90850756
90850785
106730241
175844561
175844563
175853555
175862916
175862928
175875293
175875295
175877032
175881774
175886407
175894404
175894408
175908497
175918261
175918271
175920022
175920302
175921398
175921400
175921402
175921404
175921406
175921408
175921410
175936763
175936766
175936823
175939843
175942907
175942911
175942915
175942935
175972914
175997873
176011825
176011826
176011863
176012792
176063689
176063692
176063704
176063706
176063742
176063744
176063745
176063746
176063748
176063751
176063755
176063756
176063759
176063761
176063765
176063767
176063772
176063775
176063779
176063781
176063783
176063785
176063787
176063796
176063798
176063800
176063802
176063810
176063811
176063817
176063819
176063826
176063828
176063832
176063834
176063837
176063841
176063845
176063849
176063851
176063856
176063858
176063860
176063862
176063864
176063866
176063877
176063880
176063882
176063883
176063885
176063888
176063890

In [111]:
R =BFS_res(R, 175875293)


In [112]:
R.nodes[175875293]['D']

0

In [120]:
R.nodes[176063994]['Adj']

[425402675, 176063993, 176087849, 176094665]

In [109]:
R, active = preprocess_res(R, 176063993)


In [117]:
print(R.nodes[176063993]['Adj'])

[176063994, 176063931, 176087847]


In [110]:
R.nodes[176063993]['D']

16818

In [129]:
active_nodes, R = push_relabel_res(R, 176063993, active)

In [148]:
print(R.nodes[425402675]['excess'],R[176063993][176063994]['flow'])

0 1487.4543392961573


In [151]:
R = pre_flow_push(R, 176063993)

KeyboardInterrupt: 

In [37]:
for u,v,i in denver.edges:
    edge_data = denver.get_edge_data(u, v, i)
    r = float(edge_data["capacity"])
    print(r)

5065
5585
2031
3723
3723
1287
2575
5151
1487
1487
3863
1287
4535
1487
4052
1015
3039
2031
3477
1487
1487
1487
1015
1015
1013
2318
1013
2318
1015
2031
2026
4114
3477
1403
1487
4210
2318
1487
1403
2318
1287
4210
1403
1403
1056
1287
2113
1585
2113
1487
1487
1403
1403
1487
1403
1403
1487
1403
1403
1487
2807
2807
2807
1403
1403
1487
1487
1403
1403
1403
1403
1487
1487
4210
1403
2318
1403
3509
3477
4114
2742
1287
1287
4636
1287
1287
1287
1287
1287
1287
1487
1487
1487
1287
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1403
1403
1487
1487
2105
2105
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1287
1287
1487
1487
1487
1487
1487
1487
1487
1487
3509
5614
1487
1487
1487
1487
1487
1487
1487
2974
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487


1507
2261
2261
1507
1056
1403
1056
1507
2807
1487
1487
2807
1371
1371
1487
1487
1287
1487
1287
1287
1287
1287
1487
1287
1487
1287
1487
1287
1507
1374
1487
1487
1487
1487
1487
1487
1015
1487
1507
7018
1287
1287
7018
4210
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1287
1487
1287
1487
1487
1487
1487
1487
1487
1487
1487
1507
4227
1507
1507
1507
1507
1507
1507
1287
1287
1487
1287
1507
1287
1487
1487
1487
1487
1487
4210
1487
1487
1487
1487
2807
1487
5614
3509
1487
1487
1487
1287
1287
1487
1487
4210
1403
1487
1487
1512
1512
1487
1487
1287
1287
1487
1487
4210
1487
1487
2807
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1403
1403
1487
1487
1487
1487
1487
1487
1512
1487
1487
1487
1487
1487
2897
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1403
1403
1403
1487
1403
1487
1287
1287
1487
1487
1287
5151
1487
1487
1487
1403
1403
1403
1487
1487
1403


1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1287
1287
1403
1403
1487
1487
1403
1403
1287
1287
1487
1487
1487
1487
1487
1487
1287
1287
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1403
1487
1403
1403
1403
1487
1403
1403
1487
1403
1487
1403
1487
1403
1403
1487
1403
1403
1487
1487
1403
1403
1287
1287
1403
1403
1487
1403
1403
1403
1487
1487
1403
1403
1487
1403
1403
1487
1403
1403
1487
1403
1403
1487
4210
1403
1487
1403
1403
1287
1287
1403
1403
1487
1487
1487
1487
1487
1487
1403
1403
1487
1487
4537
1403
1487
4210
1403
1487
1403
1403
1487
1403
1403
1487
1403
1403
1487
1403
1403
1487
1403
1403
1403
1403
1487
1487
1487
1487
8421
2807
1487
1487
2575
2575
2807
5151
1487
2575
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1403
1403
1403
1403
3509
3509
1487
1487
3863
10302
1487
1487
1403
1403
3509
3509
1487
1487
1487
1487
1487
1487
1403
1403
1487
1487
1487
1487
1487
1487
1487
1487
1487
1487
1403
3509
3509
3024